# AgCaP code #2
<br>

**Conceptualization, Methodology, Code:** [Davide Mazzoni](https://github.com/orgs/SEforALL-IEAP/people/davidemazzoni2) <br>  **Supervision, Review and Advisory support:** xx<br> **Funding:** SEforALL

--------------------------
## Discription of the code
This code:
- takes as input the enhanced settlement layer with all the relevant data extracted into it
- performs the required transformations
- calculates the main composite indices required for the analysis
- visualizes the results
- allows to perform a multi-criteria analysis for customized site selection and ranking processes

Note: This version takes enhanced Mozambique settlement layer with extracted attributes prepared manually in QGIS. A future verison will take the output of the main AgCAP code developed by Alexandros Korkovelos, which is meant to replace the manual QGIS data manipulation and extraction.

## Importing necessary packages

In [7]:
import geopandas as gpd
import pandas as pd
#import leafmap.kepler as leafmap
#import geemap
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
from keplergl import KeplerGl
import rasterstats
import ipywidgets as widgets
import folium
import rasterio
import os
#import panel as pn
#from bokeh.models.formatters import PrintfTickFormatter

In [8]:
ROOT_DIR = os.path.abspath(os.getcwd())
in_path = ROOT_DIR + "\\" + 'GIS_data'
workspace = ROOT_DIR + "\\" + "Output"

## Read and prepare the settlements
Read the enhanced settlements geopackage previously prepared, containing all the settlement polygons in the Country and its extracted attributes

In [11]:
ROOT_DIR

'C:\\Users\\alexl\\Dropbox\\GitHub\\AgCAP\\AgCAP_2'

In [9]:
#Read the layer already prepared by extracting all the relevant data into the clusters
# Define path and name of the file
settles_path = in_path + "\\" + "Settlements"
#settles_name = "Mozambique_Settlement_Extents_Version_02.geojson"
settles_name = "pop_clusters_enhanced_v2_Mopeia.gpkg"

## Read settlements as gdf
settl = gpd.read_file(settles_path + "\\" + settles_name)

#settl_path=r"c:/Users/david/OneDrive - Sustainable Energy for All/Cooling_module_analysis/Data/population_clusters_enhanced_v2.gpkg"
#settl=gpd.read_file(settl_path)

DriverError: C:\Users\alexl\Dropbox\GitHub\AgCAP\AgCAP_2\GIS_data\Settlements\pop_clusters_enhanced_v2_Mopeia.gpkg: No such file or directory

In [ ]:
#Transform polygons into centroids to speed up the computational time. The final results can then be joined back to the polygon layer
#settl.drop(columns=['_max','_air_2majority','_air_2mean'],inplace=True)
settl_point=settl.copy()
settl_point["geometry"]=settl_point["geometry"].centroid
settl_point

In [ ]:
settl_point.describe()

In [ ]:
settl_point.columns

In [ ]:
#Drop unnecessary columns
settl_point.drop(columns=['SHAPE_Le_1', 'SHAPE_Area','croplnd_area_m2','croplnd_area_perc','travel_ports_FAO', 'travel_cities_20k_FAO','_air_2mean', '_air_2majority', '_max'],inplace=True)

In [ ]:
# Define the column names and descriptions
data_dict = {
    'Column Name': ['country', 'iso', 'bld_count', 'dou_level1', 'dou_level2', 'pcode',
                    'area_m2', 'influence_area_m2', 'relative_wealth',
                    'livelihood_zone',
                    'food_insecur_class', 'nightlights_mean', 'nightlights_min',
                    'nightlights_max', 'electrif_status_nightlights', 'pop',
                    'fishing_activity_sea', 'inland_water_area_3km_ha',
                    'fishing_activity_inland', 'cooling_deg_days_4Cdeg',
                    'croplnd_area_GLADM19_ha', 'croplnd_area_GLADM19_perc',
                    'croplnd_3km_buff_ha_count', 'croplnd_3km_buff_ha_sum',
                    'croplnd_3km_buff_perc', 'households',
                    'food_insec_p3_plus_C_population_percentage',
                    'food_insec_p3_plus_C_population', 'mrkt_accs_cities_20kmean',
                    'mrkt_accs_ports_mean', 'dist_coast_km', 'pop_20km_buffer',
                    'travel_maj_city_200k', 'travel_to_airports', 'travel_to_railw',
                    'travel_to_capital',
                    'geometry'],
    'Description': ['Country name', 'ISO code of the country', 'Building count', 'Urbanicity level 1',
                    'Urbanicity level 2', 'Postal code', 'Settlement area in square meters',
                    'Voronoi polygon influence area in square meters',
                    'Relative wealth index', 'Livelihood zone',
                    'Food insecurity class', 'Mean nightlights value', 'Minimum nightlights value', 'Maximum nightlights value',
                    'Electrification status based on nightlights', 'Population', 'Marine fishing activity', 'Inland water area within 3km buffer, in hectares',
                    'Inland fishing activity', 'Cooling degree days at reference temperature of 4°C', 'Cropland area in hectares (GLADM 2019) within the voronoi influence area',
                    'Percentage of cropland area (GLADM 2019) within the voronoi influence area', 'Area of the polygon with a 3km buffer around the settlement', 'Sum of cropland within the 3km-buffered polygon',
                    'Percentage of cropland area within the 3km-buffered polygon', 'Number of households', 'Percentage of population with food insecurity (P3+ class)',
                    'Population with food insecurity (P3+ class)', 'Market access to closest city with population over 20,000 (FAO data)', 'Travel time to closest port (FAO data)',
                    'Distance to nearest coastline in kilometers', 'Population within 20km buffer', 'Travel time to closest major city with population over 200,000',
                    'Travel time to closest airport', 'Travel time to closest railway station', 'Travel time to capital city', 'Geometric shape of the area']
}

# Create a DataFrame from the data dictionary
data_dict = pd.DataFrame(data_dict)

# Print the DataFrame
data_dict

## Visualize the settlements
Inspect the settlement points on an interactive map

In [ ]:

settl_point.explore(
    marker_kwds=dict(radius=2, fill=True),
    style_kwds=dict(weight=0.5,opacity=0.2,fillOpacity=0.3),
    tooltip='pop',
    popup=True,  # show all values in popup (on click)
    tiles="CartoDB Positron",  # use "CartoDB positron" tiles
    tooltip_kwds={
        'style': 'font-size: 8px;'
    },
    popup_kwds={
        'style': 'font-size: 10px;'
    },
    height=1000
)

## Compute the farming activity index
1. Compute the farming activity, which is a proxy for farming activity based on:
- Farming volume: Cropland area within each settlement's influence area (Voronoi polygon)
- Farming density: Percentage of each settlement's influence area (Voronoi polygon) occupied by crops

2. Normalize the index using min-max scaling

In [ ]:
#Calculating farming activity index based on surrounding cropland area and cropland area percentage
w_croplnd_area_ha=0.5
w_croplnd_area_perc=0.5
settl_point['farming_activity']= (
    (settl_point['croplnd_area_GLADM19_ha'] - settl_point['croplnd_area_GLADM19_ha'].min())/(settl_point['croplnd_area_GLADM19_ha'].max() - settl_point['croplnd_area_GLADM19_ha'].min())*w_croplnd_area_ha
    + (settl_point['croplnd_area_GLADM19_perc'] - settl_point['croplnd_area_GLADM19_perc'].min())/(settl_point['croplnd_area_GLADM19_perc'].max() - settl_point['croplnd_area_GLADM19_perc'].min())*w_croplnd_area_perc
)
#normalize
scaler = MinMaxScaler()
settl_point['farming_activity_n']= scaler.fit_transform(settl_point[['farming_activity']])
settl_point['farming_activity_n'].plot.hist(bins=100, title='Farming Activity Index')

3. Reduce skewness to obtain a more uniform distribution

In [ ]:

#reduce skeweness
settl_point['farming_activity_n']=np.power((settl_point['farming_activity_n']),1/4)

settl_point['farming_activity_n'].plot.hist(bins=100, title='Farming Activity Index')
# Create a 3D scatter plot
# fig = px.scatter_3d(settl_point, x='croplnd_area_GLADM19_perc', y='croplnd_area_GLADM19_ha', z='farming_activity_n')
# fig.show()
# settl_point.plot.scatter(y='farming_activity_n', x='croplnd_area_GLADM19_perc')

#((settl_point['croplnd_area_GLADM19_ha'] - settl_point['croplnd_area_GLADM19_ha'].min())/(settl_point['croplnd_area_GLADM19_ha'].max() - settl_point['croplnd_area_GLADM19_ha'].min())).plot.hist(bins=100)
settl_point['farming_activity_n'].describe()

4. Visualize the farming activity index

In [ ]:
#top_farming_act=settl_point.sort_values(by='farming_activity',ascending=False)#.head(100)
top_farming_act=settl_point.loc[settl_point['farming_activity_n'] > 0.75]
m=top_farming_act.explore(
    column="farming_activity_n",
    marker_kwds=dict(radius=2, fill=True),  # make marker radius 10px with fill
    style_kwds=dict(weight=0.5,opacity=0.2,fillOpacity=0.3),  # make line thickness 1px
    tooltip="farming_activity_n",
    popup=True,  # show all values in popup (on click)
    tiles="CartoDB Positron",  # use "CartoDB positron" tiles
    cmap="viridis",  # use "Set1" matplotlib colormap
    labels=["Farming activity index"],
    legend_kwds=dict(caption='Farming activity index'),
    location=[-15.92071047059695, 35.992526466359394],
    zoom_start=7,
    #style_kwds=dict(color="black"),  # use black outline
        tooltip_kwds={
        'style': 'font-size: 8px;'
    },
    popup_kwds={
        'style': 'font-size: 10px;'
    },
    height=1000
)
m

## Compute the market accessibility index
Compute the market accessibility, which is a proxy for the possibility to trade or export agricultural products based on a weighted average of:
- travel time to the capital city (potential for export and national distribution)
- travel time to railway stations (potential for export and national distribution)
- travel time to airports (potential for export and national distribution)
- travel time to major cities (potential for sales in fresh markets and national distribution)
- population within 20 km radius (potential for sales in fresh markets)
- farming density: Percentage of each settlement's influence area (Voronoi polygon) occupied by crops
- travel time to ports (not yet implemented)

1. Normalize the accessibility parameters using min-max scaling

In [ ]:
#Normalization of accessibility parameters
""" scaler = MinMaxScaler()
settl_point['travel_to_capital_n'] = scaler.fit_transform(settl_point[['travel_to_capital']])
settl_point['travel_to_railw_n'] = scaler.fit_transform(settl_point[['travel_to_railw']])
settl_point['travel_to_airports_n'] = scaler.fit_transform(settl_point[['travel_to_airports']])
settl_point['travel_maj_city_200k_n'] = scaler.fit_transform(settl_point[['travel_maj_city_200k']])
settl_point['pop_20km_buffer_n'] = scaler.fit_transform(settl_point[['pop_20km_buffer']])
settl_point """


settl_point['travel_to_capital_n'] = (settl_point['travel_to_capital'].max()-settl_point['travel_to_capital'])/(settl_point['travel_to_capital'].max()-settl_point['travel_to_capital'].min())
settl_point['travel_to_railw_n'] = (settl_point['travel_to_railw'].max()-settl_point['travel_to_railw'])/(settl_point['travel_to_railw'].max()-settl_point['travel_to_railw'].min())
settl_point['travel_to_airports_n'] = (settl_point['travel_to_airports'].max()-settl_point['travel_to_airports'])/(settl_point['travel_to_airports'].max()-settl_point['travel_to_airports'].min())
settl_point['travel_maj_city_200k_n'] = (settl_point['travel_maj_city_200k'].max()-settl_point['travel_maj_city_200k'])/(settl_point['travel_maj_city_200k'].max()-settl_point['travel_maj_city_200k'].min())
settl_point['pop_20km_buffer_n'] = (settl_point['pop_20km_buffer']-settl_point['pop_20km_buffer'].min())/(settl_point['pop_20km_buffer'].max()-settl_point['pop_20km_buffer'].min())
pd.set_option('display.max_columns', None)
settl_point

2. Assign weights, normalize weights, calculate weighted average, normalize the index

In [ ]:
#Setting weights from 0 to 100
w_travel_to_capital=5
w_travel_to_railw=20
w_travel_to_air=50
w_travel_to_maj_city=100
w_pop20km=100

#Normalizing weights
weights=np.array([w_travel_to_capital, w_travel_to_railw, w_travel_to_air, w_travel_to_maj_city, w_pop20km])
total_weights = np.sum(weights)
w_travel_to_capital=w_travel_to_capital/total_weights
w_travel_to_railw=w_travel_to_railw/total_weights
w_travel_to_air=w_travel_to_air/total_weights
w_travel_to_maj_city=w_travel_to_maj_city/total_weights
w_pop20km=w_pop20km/total_weights

#Calculating accessibility indicator for farming activity, i.e. proxy for the ability to sell agricultural outputs
access_farm_index=(
    w_travel_to_capital*settl_point['travel_to_capital_n']+
    w_travel_to_railw*settl_point['travel_to_railw_n']+
    w_travel_to_air*settl_point['travel_to_airports_n']+
    w_travel_to_maj_city*settl_point['travel_maj_city_200k_n']+
    w_pop20km*settl_point['pop_20km_buffer_n']
)
settl_point['access_farm_index']=access_farm_index
#normalize access_farm_index
access_farm_index_n= (settl_point['access_farm_index'] - settl_point['access_farm_index'].min()) / (settl_point['access_farm_index'].max() - settl_point['access_farm_index'].min())

""" scaler = MinMaxScaler()
access_farm_index_n = scaler.fit_transform(settl_point['access_farm_index'].values.reshape(-1, 1)).flatten()
 """
settl_point['access_farm_index_n'] = access_farm_index_n
settl_point['access_farm_index_n'].plot.hist(bins=100, title='Market Accessibility Index')
#reduce skewness
#settl_point['access_farm_index_n']=np.power((settl_point['access_farm_index_n']),1)

#settl_point.head(100)

There is no need to reduce the sckewness or to further modify the distribution

Visualize the index

In [ ]:
settl_point['access_farm_index_n'].describe()

In [ ]:
top_access=settl_point.loc[settl_point['access_farm_index_n'] > 0.523573]
top_access.explore(
    column="access_farm_index_n",
    marker_kwds=dict(radius=1, fill=True),  # make marker radius 10px with fill
    style_kwds=dict(weight=0.5,opacity=0.2,fillOpacity=0.3),  # make line thickness 1px
    tooltip="access_farm_index_n",
    popup=True,  # show all values in popup (on click)
    tiles="CartoDB Positron",  # use "CartoDB positron" tiles
    cmap="viridis",  # use "Set1" matplotlib colormap
    labels=["Market Accessibility Index"],
    legend_kwds=dict(caption='Market Accessibility Index'),
       location=[-15.92071047059695, 35.992526466359394],
    zoom_start=7,
    #style_kwds=dict(color="black"),  # use black outline
            tooltip_kwds={
        'style': 'font-size: 8px;'
    },
    popup_kwds={
        'style': 'font-size: 10px;'
    },
    height=1000
                   )

## Compute the market potential index
Compute the market potential, which is a proxy for the settlement's ability to both produce and trade agricultural products, based on a weighted average of:
- farming activity
- market accessibility

In [ ]:
settl_point['mrkt_potential']=settl_point['access_farm_index_n']*0.6+settl_point['farming_activity_n']*0.4
#normalize
settl_point['mrkt_potential_n'] = scaler.fit_transform(settl_point['mrkt_potential'].values.reshape(-1, 1)).flatten()
settl_point['mrkt_potential_n'].plot.hist(bins=100, title='Market Potential Index (farming activity + market accessibility)')
settl_point.plot.scatter(y='mrkt_potential_n', x='farming_activity_n')
settl_point.plot.scatter(y='mrkt_potential_n', x='access_farm_index_n')

In [ ]:
settl_point['mrkt_potential_n'].describe()

Visualize the Market Potential Index

In [ ]:
#top_farming_mrkt_pot=settl_point.sort_values(by='mrkt_potential_n',ascending=False).head(1000)
settl_point['mrkt_potential_n'].describe()

top_farming_mrkt_pot=settl_point.loc[settl_point['mrkt_potential_n'] > .550311]
m=top_farming_mrkt_pot.explore(
    column='mrkt_potential_n',
    marker_kwds=dict(radius=2, fill=True),  # make marker radius 10px with fill
    style_kwds=dict(weight=0.5,opacity=0.2,fillOpacity=0.3),  # make line thickness 1px
    tooltip='mrkt_potential_n',
    popup=True,  # show all values in popup (on click)
    tiles="CartoDB Positron",  # use "CartoDB positron" tiles
    cmap="viridis",  # use "Set1" matplotlib colormap
    labels=["Farming market potential"],
    legend_kwds=dict(caption='Farming market potential (farming activity + access to markets)'),
    location=[-15.92071047059695, 35.992526466359394],
    zoom_start=7,
    #style_kwds=dict(color="black"),  # use black outline
                tooltip_kwds={
        'style': 'font-size: 8px;'
    },
    popup_kwds={
        'style': 'font-size: 10px;'
    },
    height=1000
)
m
#top_farming_mrkt_pot.explore(tooltip=['mrkt_potential_n','access_farm_index_n','farming_activity_n'])

## Pending modules
- Crop-specific analysis, including insights on high-value crops or specific value chains
- Fish market potential
- Livestock market potential
- Climate risk indicators, for site selection based on climate adaptation needs
- More robust socio-economic indicator (currently based on Relative Wealth Index)
- More refined electrification status indicator, based on distance from MV lines and night ligthts (currently only based on night lights)

## Create multi-criteria selection tool (Beta)

In [ ]:
w=widgets.FloatRangeSlider(
    value=[0, 6],
    min=0, #settl_point['travel_to_airports'].min(),
    max=9, #settl_point['travel_to_airports'].max(),
    step=0.1,
    description='travel_to_airports',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [ ]:
w.value

In [ ]:
pn.extension()
slider_pop = pn.widgets.EditableRangeSlider(
    name='Population', start=0, end=2000000, value=(5000,80000),
    step=50)

slider_farm_act = pn.widgets.EditableRangeSlider(
    name='Farming activity index', start=0, end=1, value=(0.6,1),
    step=50)

slider_fish_act_marine = pn.widgets.EditableRangeSlider(
    name='Marine fishing activity index', start=0, end=1, value=(0.6,1),
    step=0.01)

slider_fish_act_inland = pn.widgets.EditableRangeSlider(
    name='Inland fishing activity index', start=0, end=1, value=(0.6,1),
    step=0.01)

checkbox_group_electr_mk = pn.pane.Markdown('Electrification status')
checkbox_group_electr = pn.widgets.CheckBoxGroup(
    name='Electrification', value=['Not electrified'], options=['Electrified', 'Likely electrified', 'Not electrified'],
    inline=True)

slider_dist_grid= pn.widgets.EditableRangeSlider(
    name='Distance from the grid MV lines (km)', start=0, end=1000, value=(0,1.5),
    step=0.1)

slider_pop_nearby= pn.widgets.EditableRangeSlider(
    name='Local market: population within 20 km radius', start=0, end=2700000, value=(0,2700000),
    step=50)

slider_tt_airport= pn.widgets.EditableRangeSlider(
    name='Travel time to airports (hr)', start=0, end=20, value=(0,5),
    step=0.1)

slider_tt_maj_city= pn.widgets.EditableRangeSlider(
    name='Travel time to major cities (hr)', start=0, end=20, value=(0,2),
    step=0.1)

slider_tt_railw= pn.widgets.EditableRangeSlider(
    name='Travel time to railway stations (hr)', start=0, end=20, value=(0,1),
    step=0.1)

slider_tt_port= pn.widgets.EditableRangeSlider(
    name='Travel time to ports (hr)', start=0, end=20, value=(0,2),
    step=0.1)

slider_tt_capital= pn.widgets.EditableRangeSlider(
    name='Travel time to Maputo (hr)', start=0, end=20, value=(0,6),
    step=0.1)

slider_poverty= pn.widgets.EditableRangeSlider(
    name='Poverty rate', start=0, end=1, value=(0.3,1),
    step=0.01)

slider_temp= pn.widgets.EditableRangeSlider(
    name='Average temperature', start=10, end=40, value=(22,35),
    step=0.1)

slider_CDD=pn.widgets.EditableRangeSlider(
    name='Cooling degree days @ 4C', start=4108, end=8413, value=(7000,8413),
    step=0.1)

slider_risk= pn.widgets.MultiChoice(
    name='Climate risk', value=['Low', 'Medium', 'High'] ,options=['Low', 'Medium', 'High'])

slider_solar= pn.widgets.MultiChoice(
    name='Solar irradiation (for solar-powered cold rooms)', value=['Low', 'Medium', 'High'] ,options=['Low', 'Medium', 'High'])

checkbox_group_produce = pn.widgets.MultiChoice(
    name='Producer of:',value=['banana',
'cassava',
'cowpea',
'other roots',
'plantain',
'potato',
'sweet potato',
'temperate fruit',
'tropical fruit',
'vegetables',
'yams',
'meat',
'fish',
],
    options=['banana',
'cassava',
'cowpea',
'other roots',
'plantain',
'potato',
'sweet potato',
'temperate fruit',
'tropical fruit',
'vegetables',
'yams',
'meat',
'fish',
],
   )

pn.config.sizing_mode='scale_both'
""" pane_inputs=pn.Column(
            slider_pop, 
          slider_farm_act,
          slider_fish_act_marine,
          slider_fish_act_inland,
          checkbox_group_electr_mk,
          checkbox_group_electr,
          checkbox_group_produce,
          slider_pop_nearby,
          slider_tt_airport,
          slider_tt_maj_city,
          slider_tt_railw,
          slider_tt_port,
          slider_tt_capital,
          slider_poverty,
          slider_risk,
          slider_solar
          ) """
          
pane_inputs=pn.Row(
    pn.layout.Divider(margin=0),
    pn.Column(
            '## Demographics ans socio-economic',
            slider_pop, 
            slider_poverty,
            "## Composite indicators",
          slider_farm_act,
          slider_fish_act_marine,
          slider_fish_act_inland,
          ".../n",
          "## Electrification",
          checkbox_group_electr_mk,
          checkbox_group_electr,
          slider_dist_grid,
          "## Productivity",
          "... production (tons) per produce",
          checkbox_group_produce),

    pn.layout.Divider(margin=(30,0,0,0)),
     pn.Column(
          "## Access to markets",
          slider_pop_nearby,
          slider_tt_airport,
          slider_tt_maj_city,
          slider_tt_railw,
          slider_tt_port,
          slider_tt_capital,
          "## Risks",
          slider_risk,
          "## Resources",
          slider_solar,
          "## Temperature / cooling needs",
          slider_temp,
          slider_CDD
          )
    )
pane_inputs

Save pane in HTML (optional)

In [ ]:
#pn.panel(pane_inputs).save('pane_3.html')

Export geopackage (optional)

In [ ]:
#settl_point.to_file('settl_point_farm_ind.gpkg', driver='GPKG')